# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2024-07-21 14:07:33.454178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [14]:
"""
apply transformtions to prevent overfitting -> geometrical transformations like zooms, rotations, etc 
to make the images more diverses -> this is called augmentation
"""

train_datagen = ImageDataGenerator(rescale = 1./255, # feature scaling
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# flow_from_directory = connects the augmentation class with the images dataset
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # less target_size = more fast training
                                                 batch_size = 32,
                                                 class_mode = 'binary') # binary as outout can be cat or dogs

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [15]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [16]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [17]:
"""
input shape = target_size in the flow_from_directory,
we put 3 at the end because we are working with color images,
if it was black/white images, this would be 1
"""

convolution = tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 3, # means a matrix 3x3
    activation = 'relu', 
    input_shape = [64, 64, 3]
)

# add the layer
cnn.add(convolution)

### Step 2 - Pooling

In [18]:
# to get the pooled feature map, getting the max pixels when applying the kernel

"""
pool_size = max pixel mapping by 2x2
strides = how many pixels we move along when applying the pool size kernel
"""
pooling = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)

cnn.add(pooling)

### Adding a second convolutional layer

In [19]:
convolution = tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    activation = 'relu',
)

# add the layer
cnn.add(convolution)

pooling = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)

# add the layer
cnn.add(pooling)

### Step 3 - Flattening

In [20]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [21]:
# neurons layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [22]:
# only need one neuron as binary classification
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [23]:
cnn.compile(
    optimizer ='adam', # stochastic gradient descent 
    loss= 'binary_crossentropy',
    metrics = ['accuracy']
)

### Training the CNN on the Training set and evaluating it on the Test set

In [24]:
cnn.fit(
    x=training_set,
    validation_data=test_set,
    epochs = 25
)

Epoch 1/25


2024-07-15 21:32:29.244013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - ETA: 0s - loss: 0.6672 - accuracy: 0.5876

2024-07-15 21:32:58.452982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - 34s 131ms/step - loss: 0.6672 - accuracy: 0.5876 - val_loss: 0.6297 - val_accuracy: 0.6630
Epoch 2/25
250/250 [==============================] - 33s 133ms/step - loss: 0.6023 - accuracy: 0.6718 - val_loss: 0.5897 - val_accuracy: 0.6880
Epoch 3/25
250/250 [==============================] - 34s 136ms/step - loss: 0.5678 - accuracy: 0.7028 - val_loss: 0.5272 - val_accuracy: 0.7530
Epoch 4/25
250/250 [==============================] - 34s 136ms/step - loss: 0.5385 - accuracy: 0.7318 - val_loss: 0.5079 - val_accuracy: 0.7520
Epoch 5/25
250/250 [==============================] - 37s 146ms/step - loss: 0.5103 - accuracy: 0.7464 - val_loss: 0.5084 - val_accuracy: 0.7630
Epoch 6/25
250/250 [==============================] - 35s 139ms/step - loss: 0.4911 - accuracy: 0.7590 - val_loss: 0.4888 - val_accuracy: 0.7630
Epoch 7/25
250/250 [==============================] - 40s 160ms/step - loss: 0.4746 - accuracy: 0.7724 - val_loss: 0.5010 - val_accuracy: 0.7

## Part 4 - Making a single prediction

In [34]:
import numpy as np
import keras.utils as image
# from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)

# adding it to a batch because the cnn understand in term of batches not single images (a batch can have just one image)
test_image = np.expand_dims(test_image, axis = 0) #

result = cnn.predict(test_image)

# 
training_set.class_indices

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
  

1/1 [==============================] - 0s 21ms/step


In [35]:
print(prediction)

dog
